$$\frac{{\rm d} N}{{\rm d} t} = r N \left(1 - \frac{N}{K}\right) - \frac{cN^2}{H^2 + N^2} + \sigma {\rm d}W_t.$$

We will integrate this equation using Euler's method and add a stochastic term to this model (the hand that rocks the cradle), in order to find equilibria and/or bifurcation points. This will generate some time-series that we can analyse subsequently.

$$N_{i+1} = N_{i} + \Delta{}t \left(r N_{i} \left(1 - \frac{N_{i}}{K}\right) - \frac{c N_{i}^2}{H^2 + N_{i}^2}\right) + \sigma\ \Delta{}W_i$$



> #### Note: complex numbers
> Note that Cardano's formula involves complex numbers. The Python syntax for imaginary numbers is the floating point value with a `j` suffix.
>
> ```python
> >>> x = 1 + 2j
> >>> x.real
> 1.0
> >>> x.imag
> 2.0
> >>> print(x)
> (1+2j)
```
>
> Computing square roots using the power notation ``a**b`` automatically results in complex values when the need arises:
> 
> ```python
> >>> from numpy import sqrt, cbrt
> >>> sqrt(-1)
> __main__:1: RuntimeWarning: invalid value encountered in sqrt
> nan
> >>> (-1)**(1/2)
> (6.123233995736766e-17+1j)
> >>> sqrt(-1+0j)
> 1j
> ```

> #### Note: Documentation
> The first expression in the definition of `cubic_roots` is a long string containing the documentation of the function: the *docstring*. All parameters to the function, and its return value are touched upon. You can look up the docstring of a function or object by typing `help(<function name>)`.
>
> For the curious: the particular format of this docstring is not set in stone, but in this case we use [Sphinx Restructured Text](http://www.sphinx-doc.org/en/stable/rest.html). Sphinx enables us to create online documentation for libraries.

In [1]:
def simple_moving_average(y, N):
    y_padded = np.r_[np.repeat(y[0], N), y]
    cs = np.cumsum(y_padded)
    return (cs[N:] - cs[:-N]) / N

def centered_moving_average(y, N):
    y_padded = np.r_[np.repeat(y[0], (N+1)//2), y, np.repeat(y[-1], N//2)]
    cs = np.cumsum(y_padded)
    return (cs[N:] - cs[:-N]) / N

def smooth(y, N, window, **kwargs):
    y_padded = np.r_[np.repeat(y[0], N//2), y, np.repeat(y[-1], N//2)]
    w = getattr(np, window)(N, **kwargs)
    w /= w.sum()
    return np.convolve(y_padded, w, mode='valid')

In [ ]:
y = x[:,0]
y_sma_10 = simple_moving_average(y, 2001)
y_cma_10 = centered_moving_average(y, 2001)
y_smooth = smooth(y, 2001, window='hamming')

fig1 = plt.subplot(231)
fig1.plot(t, y)
fig1.plot(t, y_sma_10)
fig2 = plt.subplot(234)
fig2.plot(t, y - y_sma_10)

fig3 = plt.subplot(232)
fig3.plot(t, y)
fig3.plot(t, y_cma_10)
fig4 = plt.subplot(235)
fig4.plot(t, y - y_cma_10)

fig5 = plt.subplot(233)
fig5.plot(t, y)
fig5.plot(t, y_smooth)
fig6 = plt.subplot(236)
fig6.plot(t, y - y_smooth)

plt.show()

In [ ]:
# compute moving std
res = y - y_smooth
idx = np.arange(1000)[None,:] + np.arange(9000)[:, None]
stds = res[idx].std(axis=1)

In [ ]:
from scipy.ndimage import filters

from scipy import signal

y = x[:, 0]
widths = np.logspace(1.0, 3.0, 256)
res = y - simple_moving_average(y, 2001)
res_cwt = signal.cwt(res[:13000], signal.ricker, widths)

extrema = np.where(np.logical_or(
    filters.maximum_filter(res_cwt, size=10, mode='nearest') == res_cwt,
    filters.minimum_filter(res_cwt, size=10, mode='nearest') == res_cwt))

good = np.where(
    np.logical_and(
        np.logical_and(extrema[0]!=0, extrema[0]!=res_cwt.shape[0]-1),
        np.logical_and(extrema[1]!=0, extrema[1]!=res_cwt.shape[1]-1)))

extreme_values = np.c_[t[extrema[1][good]], widths[extrema[0][good]], res_cwt[extrema[0][good], extrema[1][good]]**2]

fig = plt.subplot(111)
fig.set_yscale('log')
fig.pcolormesh(t[:13000], widths, res_cwt, cmap='RdYlBu')
fig.plot(extreme_values[:,0], extreme_values[:,1], '.', c='k')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1)

for i in range(m):
    y = x[:, i]
    widths = np.logspace(1.0, 3.0, 256)
    res = y - simple_moving_average(y, 5001)
    res_cwt = signal.cwt(res[:13000], signal.ricker, widths)

    extrema = np.where(np.logical_or(
        filters.maximum_filter(res_cwt, size=10, mode='nearest') == res_cwt,
        filters.minimum_filter(res_cwt, size=10, mode='nearest') == res_cwt))

    good = np.where(
        np.logical_and(
            np.logical_and(extrema[0]!=0, extrema[0]!=res_cwt.shape[0]-1),
            np.logical_and(extrema[1]!=0, extrema[1]!=res_cwt.shape[1]-1)))

    extreme_values = np.c_[
        t[extrema[1][good]],
        widths[extrema[0][good]],
        res_cwt[extrema[0][good], extrema[1][good]]]

    ax.scatter(
        extreme_values[:,0],
        extreme_values[:,2]**2 / extreme_values[:,1],
        s=extreme_values[:,1] / 5)

ax.set_xlabel('t')
ax.set_ylabel('peak size')
ax.set_ylim(0.0, 1.0)
plt.show()

In [ ]:
def euler_maruyama(df, dt, x0, sigma, args, n, force_positive=True):
    """Integrate a stochastic differential equation.
    
    :param df: Function that gives dx/dt as function of x and t, non-stochastic part.
    :param dt: time-step.
    :param x0: initial values, should be array.
    :param sigma: scale factor function sigma(x, t) for stochastic term.
    :param args: extra keyword arguments for `df`.
    :param n: number of steps to integrate.
    :return: 2d array with time-series for each given initial value.
    """
    m = x0.size                  # number of simulations
    x = np.zeros(shape=(n, m))   # storage for results
    x[0] = x0                    # assign result for t=0
    t = np.arange(n) * dt        # define time values

    def dW():
        return np.random.normal(loc=0.0, scale=np.sqrt(dt), size=m)
    
    for i in range(0, n-1):
        x[i+1] = x[i] + dt * df(x[i], t[i], **args) + sigma(x[i], t[i]) * dW()
        if force_positive:
            x[i+1] = np.where(x[i+1] < 0, 0.0, x[i+1])
    
    return t, x

We have written the stochastic term ${\rm d}W$ as a function inside a function. The reason that we did this is that the most important line in the implementation,
```python
x[i+1] = x[i] + dt * df(x[i], t[i], **args) + sigma(x[i], t[i]) * dW()
```
now reflects the mathematical description of the Euler-Maruyama integrator,

$$x_{i+1} = x_i + f'(x_i, t_i)\ \Delta{}t + \sigma(x_i, t_i)\ \Delta{}W_i.$$

This makes the code more readable, which is a *good thing*.


In [ ]:
sigma_const = 0.1
sigma = lambda x, t: sigma_const
m = 50
x0 = np.linspace(0, 10, m)


for idx, c in enumerate([1.6, 1.787, 2.604, 2.8]):
    settings.update(c=c)
    t, x = euler_maruyama(
        df=dNdt, dt=0.1, x0=x0, sigma=sigma, args=settings, n=500)
    
    fig = plt.subplot(220 + idx + 1)
    fig.set_ylim(0.0, 10)
    fig.plot(np.repeat(t[:,None], m, axis=1), x)
    fig.set_title('c={:.03}'.format(c))

plt.show()